In [1]:
#! pip install beautifulsoup4



In [2]:
page = 'https://www.stepstone.de'

In [3]:
langueList =  ['Java'.lower(), 'C#'.lower() , 'C++'.lower() , 'html'.lower(),'Python'.lower(),'C'.lower() ,
               'PHP'.lower(), 'JavaScript'.lower(), 'SQL'.lower()]
print (langueList)


def createPointList():
    pointList =[]
    for item in langueList:
        pointList.append(0)
    return pointList

def createFoundList():
    foundList =[]
    for item in langueList:
        foundList.append(False)
    return foundList    
print (createFoundList())
pointList = createPointList()
print( pointList)    

['java', 'c#', 'c++', 'html', 'python', 'c', 'php', 'javascript', 'sql']
[False, False, False, False, False, False, False, False, False]
[0, 0, 0, 0, 0, 0, 0, 0, 0]


In [5]:
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import lxml.html as lh
def getReport(url):
    global page
    
    urlList = getUrlFromWebsite(url)
    for link in urlList:
        text = getItemsFromURL(page+link)
        #print(urlList)
        analyseText(text)
    
    #getNames:
    #max_year = datetime.now().year - 1

    
       # https://www.indeed.com/jobs?q=informatik&l=
        #url = 'https://www.indeed.com/jobs?q=informatik&l='
        #url ='https://www.indeed.com/q-informatik-jobs.html?vjk=b10565c7de7d9d1c'
       # url = 'https://jobboerse.arbeitsagentur.de/vamJB/stellenangeboteFinden.html?execution=e1s1'
        #url = 'https://www.stepstone.de/5/ergebnisliste.html?stf=freeText&ns=1&qs=%5B%7B%22id%22%3A%22216805%22%2C%22description%22%3A%22Software-Entwickler%2Fin%22%2C%22type%22%3A%22jd%22%7D%5D&companyID=0&cityID=0&sourceOfTheSearchField=resultlistpage%3Ageneral&searchOrigin=Resultlist_top-search&ke=Software-Entwickler%2Fin&ws=&ra=30'
        #url = 'https://www.stepstone.de/stellenangebote--Software-Entwickler-Android-m-w-d-Darmstadt-Deutsche-Telekom-AG--6341787-inline.html?rltr=1_1_25_dynrl_m_0_1_0'
        #url = 'https://www.ssa.gov/cgi-bin/popularnames.cgi'
        #url = 'https://www.stepstone.de/stellenangebote--Software-Entwickler-Android-m-w-d-Darmstadt-Deutsche-Telekom-AG--6341787-inline.html?rltr=1_1_25_dynrl_m_0_1_0'
        #page = requests.post(url)#, {'year' : year,'top' : anzahl, 'number' : 1  })

        # Analysiere den HTML-Code
       # doc = lh.fromstring(page.content)
        #soup = BeautifulSoup(page.content, 'html.parser')
        #week = soup.find_all('a')
        
        #week = soup.find_all(class_ = 'card__body richtext')
       # print (soup)
        #return getItemsFromURL(url)
       
       # print (week)
def getItemsFromURL(url):
       
        page = requests.post(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        week = []
        
        #der wichtigen teile der Jobangebote werden extrahiert
        week = soup.find_all(class_ = 'card__body richtext')
        text = ''
        for i  in range (0,len(week)) :
            #an der stelle wird davon ausgegangen dass die vorraussetung in stichpunkten angelegt sind 
            #ist eine gewagte these aber in der regel wahr
            text += str(week[i].find_all('li'))
            #print (week[i].find_all('li'))
        #items = week[2].find_all('li')
        #print (items)
       # print (text)
        #störende zeichen die das Ergebnis verfäschen können werden entfernt
        #print(text)
        text = text.replace('<li>',' ')
        text = text.replace('</li>',' ')
        text = text.replace(',',' ')
        text = text.replace('-',' ')
        text = text.replace('/',' ')
        return text
    #    Ausdruck 	Funktion
#A 	Findet alle Vorkommen des Zeichens A im String.
#(A|B) 	Findet alle Stellen im String, die dem Zeichen A oder dem Zeichen B entsprechen.
#(...) 	Definiert eine Gruppe. Innerhalb von Gruppen werden Zeichenabfolgen in der vorgegebenen Reihenfolge gematcht. Die runden Klammern sind Teil der Syntax von regulären Ausdrücken und finden sich nicht im String wieder.
#\( und \) 	Matcht auf öffnende bzw. schließende runde Klammern im String.
#[...] 	Definiert eine Menge. Die Reihenfolge der Zeichen spielt keine Rolle. Jedes Vorkommen von einem der Zeichen in der Menge führt zu einem Match. Groß- und Kleinschreibung wird unterschieden!
#[A-Z] 	Definiert die Menge aller Großbuchstaben von A bis Z (beide inklusive).
#[e-h] 	Definiert die Menge aller Kleinbuchstaben von e bis h (beide inklusive).
#[^...] 	Definiert eine auszuschließende Menge. Ein Match findet an jeder Stelle des Strings statt, an der keins der Zeichen in der Menge steht.
#\[ und \] 	Matcht auf öffnende bzw. schließende eckige Klammern im String.

def getUrlFromWebsite(url):
    page = requests.post(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    urlList = []
    for a in soup.find_all('a', href=True):
        if ('stellenangebote' in str(a)):
            urlList.append (a['href'])
    return urlList    
    #print (urlList)
    
   # return 0
    

def analyseText(text):
    global langueList , pointList
    print (text)
    foundList = createFoundList()
    for word in str(text).split(' ') :
        #print(word)
        if(word.lower() in langueList):
            for i in range (len(langueList)):
                if (langueList[i] == word.lower() and foundList[i] == False):
                    pointList[i] = pointList[i] +1 
                    foundList[i] = True               
    print(pointList )
    #print(text)
    
#url = 'https://www.stepstone.de/stellenangebote--Software-Entwickler-Android-m-w-d-Darmstadt-Deutsche-Telekom-AG--6341787-inline.html?rltr=1_1_25_dynrl_m_0_1_0'
url = 'https://www.stepstone.de/stellenangebote--Software-Entwickler-Applikation-m-w-d-Darmstadt-Deutsche-Telekom-AG--6341089-inline.html?rltr=3_3_25_dynrl_m_0_5_0'               
text = getItemsFromURL(url)
analyseText(text)
url2 = 'https://www.stepstone.de/5/ergebnisliste.html?stf=freeText&ns=1&qs=%5B%7B%22id%22%3A%22216805%22%2C%22description%22%3A%22Software-Entwickler%2Fin%22%2C%22type%22%3A%22jd%22%7D%5D&companyID=0&cityID=0&sourceOfTheSearchField=resultlistpage%3Ageneral&searchOrigin=Resultlist_top-search&ke=Software-Entwickler%2Fin&ws=&ra=30'
#getUrlFromWebsite(url2)
getReport(url2)
print (pointList)
#b10565c7de7d9d1c
#3907019553144236&vjk=be6d77bdaaa5c61f



[0, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 0, 0, 0, 0, 0, 0, 0, 0]


KeyboardInterrupt: 